In [4]:
# simulate the FIFA World cup 10000 times.

from collections import namedtuple
from itertools import chain, groupby
import csv
import numpy

Match = namedtuple("Match", ['gameId', 'date', 'time', 'team1', 'team2', 'group'])
Prediction = namedtuple("Prediction", ['team1', 'team2', 'outcome'])
Team = namedtuple("Team", ['name', 'draw'])
Result = namedtuple("Result", ['match', 'team1Score', 'team2Score'])
TeamResult = namedtuple("TeamResult", ['name', 'opponent', 'group', 'gs', 'ga'])
TeamRecord = namedtuple("TeamRecord", ['name', 'group', 'p', 'gs', 'ga', 'defeated'])

def loadCsv(filepath, itemtype):
    with open(filepath, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        return list(map(lambda row: itemtype(**row), reader))

def getActualGroupMatch(match, teams):
    return Match(gameId = match.gameId, date = match.date, time = match.time, group = match.group,
        team1 = next(team.name for team in teams if team.draw == match.team1),
        team2 = next(team.name for team in teams if team.draw == match.team2),       
        )

def getActualEliminationMatch(match, qualified):
    return Match(gameId = match.gameId, date = match.date, time = match.time, group = match.group,
        team1 = qualified[match.team1],
        team2 = qualified[match.team2],
        )

def getResult(match, predictions):
    allowtie = match.group != None 
    prediction = next(pred for pred in predictions \
        if (pred.team1 == match.team1 and pred.team2 == match.team2) or \
            (pred.team1 == match.team2 and pred.team2 == match.team1)) 
    reverse = prediction.team1 != match.team1
    goaldiff = numpy.random.normal(float(prediction.outcome))
    reverse = reverse != (goaldiff < 0) 
    normgoaldiff = 0 if abs(goaldiff) <= 0.4475 \
        else 1 if abs(goaldiff) < 1 else round(abs(goaldiff))
    if not(allowtie) and normgoaldiff == 0:
        normgoaldiff = 1
    losergoals = round(abs(numpy.random.normal(0, 1)))
    return Result(match, losergoals, losergoals + normgoaldiff) if reverse \
        else Result(match, losergoals + normgoaldiff, losergoals)

def getTeamPerfFromResult(result):
    return [TeamResult(name=result.match.team1, opponent=result.match.team2, group=result.match.group, gs=result.team1Score, ga=result.team2Score),
    TeamResult(name=result.match.team2, opponent=result.match.team1, group=result.match.group, gs=result.team2Score, ga=result.team1Score)]

def getTeamRecord(name, teamResults):
    points = w = d = gs = ga = 0
    group = ''
    defeated = []
    for result in teamResults:
        group = result.group
        gs += result.gs
        ga += result.ga
        if result.gs > result.ga:
            w += 1
            defeated.append(result.opponent)
        elif result.gs == result.ga:
            d += 1
    return TeamRecord(name=name, group=group, p=3*w+d, gs=gs, ga=ga, defeated=defeated)

def runGroupStage(schedule, predictions, teams):
    groupschedule = list(map(lambda x: getActualGroupMatch(x, teams), filter(lambda x: x.group != None, schedule)))
    groupresults = list(map(lambda x: getResult(x, predictions), groupschedule))
    groupperfs = list(chain.from_iterable(map(getTeamPerfFromResult, groupresults)))
    records = []
    for k, g in groupby(sorted(groupperfs, key=lambda x: x.name), key=lambda x: x.name):
        records.append(getTeamRecord(k, g))
    qualified = {}
    for k, g in groupby(sorted(records, key=lambda x: x.group), key=lambda x: x.group):
        grouptable = sorted(g, key=lambda x: 1000000*x.p + 10000*(x.gs-x.ga) + 100*(x.gs), reverse=True)
        qualified['1' + k] = grouptable[0].name
        qualified['2' + k] = grouptable[1].name
    return groupresults, qualified

def runEliminationRound(round, prevround, schedule, predictions):
    roundschedule = list(map(lambda x: getActualEliminationMatch(x, prevround), filter(lambda x: x.gameId.startswith('G'+round), schedule)))
    roundresults = list(map(lambda x: getResult(x, predictions), roundschedule))
    results = {}
    for result in roundresults:
        results['W' + result.match.gameId] = result.match.team1 if result.team1Score > result.team2Score \
            else result.match.team2
        results['L' + result.match.gameId] = result.match.team1 if result.team1Score < result.team2Score \
            else result.match.team2
    return roundresults, results

#main
schedule = loadCsv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\wc2018schedule.csv', Match)
predictions = loadCsv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\predictionsfinal.csv', Prediction)
teams = loadCsv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\wc2018qualified.csv', Team)
#f = open("simresultsfinal.csv", "w+")
#f.write('iteration,winner\n')
#for i in range(20000):
#    results = runGroupStage(schedule, predictions, teams)
#    for rnd in ['AF','QF','SF','FN']:
#        results = runEliminationRound(rnd, results, schedule, predictions)

#    f.write(str(i+1) + ',' + results['WGFN2'] + '\n')

In [33]:
#for i in range(100):
#    groups = runGroupStage(schedule, predictions, teams)

    #print(str(i+1) + ',' + groups + '\n')

import pandas as pd
groupstages = pd.DataFrame()
qualified1 = pd.DataFrame()

for i in range(5):
    results = runGroupStage(schedule, predictions, teams)
    groupstages = groupstages.append(pd.DataFrame(records))
    #groupstages.append(pd.DataFrame(records))
    for rnd in ['AF','QF','SF','FN']:
        results = runEliminationRound(rnd, results, schedule, predictions)

from collections import Counter

groupstages
#Counter(groupstages.values())


TypeError: tuple indices must be integers or slices, not str

In [37]:
for i in range(10):
    groupresults, qualified = runGroupStage(schedule, predictions, teams)
    print(qualified)
    for rnd in ['AF','QF','SF','FN']:
        winners = runEliminationRound(rnd, qualified, schedule, predictions)
        print(winners)


{'1A': 'URU', '2A': 'RUS', '1B': 'IRN', '2B': 'ESP', '1C': 'AUS', '2C': 'FRA', '1D': 'ISL', '2D': 'NGA', '1E': 'SRB', '2E': 'SUI', '1F': 'MEX', '2F': 'KOR', '1G': 'BEL', '2G': 'ENG', '1H': 'SEN', '2H': 'JPN'}
{'WGAF1': 'AUS', 'LGAF1': 'NGA', 'WGAF2': 'ESP', 'LGAF2': 'URU', 'WGAF3': 'IRN', 'LGAF3': 'RUS', 'WGAF4': 'ISL', 'LGAF4': 'FRA', 'WGAF5': 'KOR', 'LGAF5': 'SRB', 'WGAF6': 'BEL', 'LGAF6': 'JPN', 'WGAF7': 'MEX', 'LGAF7': 'SUI', 'WGAF8': 'SEN', 'LGAF8': 'ENG'}


KeyError: 'WGAF1'

In [164]:
tester, tester1 = runGroupStage(schedule, predictions, teams)
tester

[Result(match=Match(gameId='GA1', date='14/06/2018', time='16:00', team1='RUS', team2='KSA', group='A'), team1Score=0, team2Score=1),
 Result(match=Match(gameId='GA2', date='15/06/2018', time='13:00', team1='EGY', team2='URU', group='A'), team1Score=1, team2Score=1),
 Result(match=Match(gameId='GA3', date='19/06/2018', time='19:00', team1='RUS', team2='EGY', group='A'), team1Score=0, team2Score=2),
 Result(match=Match(gameId='GA4', date='20/06/2018', time='16:00', team1='URU', team2='KSA', group='A'), team1Score=1, team2Score=1),
 Result(match=Match(gameId='GA5', date='25/06/2018', time='15:00', team1='URU', team2='RUS', group='A'), team1Score=1, team2Score=0),
 Result(match=Match(gameId='GA6', date='25/06/2018', time='15:00', team1='KSA', team2='EGY', group='A'), team1Score=1, team2Score=0),
 Result(match=Match(gameId='GB1', date='15/06/2018', time='16:00', team1='MAR', team2='IRN', group='B'), team1Score=0, team2Score=0),
 Result(match=Match(gameId='GB2', date='15/06/2018', time='19:

In [49]:
df1 = pd.DataFrame()
for i in range(10000):
    groupresults_y, qualified_y = runGroupStage(schedule, predictions, teams)
    df1 = df.append(test)
    
df1['score'] = df1['team1Score'].map(str) + '-' + df1['team2Score'].map(str)

df1_grouped = df1.groupby('match')['score'].value_counts()
df1_grouped2 = df1_grouped.groupby('match').idxmax()
#display(tester.loc[tester.score.max()])
df1_grouped2 = pd.DataFrame(df1_grouped2)
pd.set_option('display.max_rows', 500)
df1_grouped2

,score
match,
"(GA1, 14/06/2018, 16:00, RUS, KSA, A)","((GA1, 14/06/2018, 16:00, RUS, KSA, A), 2-1)"
"(GA2, 15/06/2018, 13:00, EGY, URU, A)","((GA2, 15/06/2018, 13:00, EGY, URU, A), 2-1)"
"(GA3, 19/06/2018, 19:00, RUS, EGY, A)","((GA3, 19/06/2018, 19:00, RUS, EGY, A), 1-1)"
"(GA4, 20/06/2018, 16:00, URU, KSA, A)","((GA4, 20/06/2018, 16:00, URU, KSA, A), 1-2)"
"(GA5, 25/06/2018, 15:00, URU, RUS, A)","((GA5, 25/06/2018, 15:00, URU, RUS, A), 1-2)"
"(GA6, 25/06/2018, 15:00, KSA, EGY, A)","((GA6, 25/06/2018, 15:00, KSA, EGY, A), 1-2)"
"(GB1, 15/06/2018, 16:00, MAR, IRN, B)","((GB1, 15/06/2018, 16:00, MAR, IRN, B), 1-2)"
"(GB2, 15/06/2018, 19:00, POR, ESP, B)","((GB2, 15/06/2018, 19:00, POR, ESP, B), 1-1)"
"(GB3, 20/06/2018, 13:00, POR, MAR, B)","((GB3, 20/06/2018, 13:00, POR, MAR, B), 2-1)"


In [198]:
grptest = pd.DataFrame()
#for i in range(10):
 #   qualified = runGroupStage(schedule, predictions, teams)
 #   grptest = grptest.append(qualified, ignore_index=True)



df2 = pd.DataFrame()
for i in range(10000):
    groupresults_x, qualified_x = runGroupStage(schedule, predictions, teams)
    df2 = df.append(groupresults_x)
    
df2['score'] = df['team1Score'].map(str) + '-' + df['team2Score'].map(str)

df1_grouped = df2.groupby('match')['score'].value_counts()
df1_grouped2 = df1_grouped.groupby('match').idxmax()
#display(tester.loc[tester.score.max()])
df1_grouped2 = pd.DataFrame(test)
pd.set_option('display.max_rows', 500)
df1_grouped2

score
match                                 score       
(GA1, 14/06/2018, 16:00, RUS, KSA, A) 1-1     3316
                                      1-2     2654
                                      0-0     2618
                                      2-1     2338
                                      0-1     2151
                                      1-0     1858
                                      2-2      827
                                      2-3      673
                                      1-3      655
                                      3-2      602
                                      0-2      570
                                      3-1      523
                                      2-0      418
                                      2-4      167
                                      4-2      143
                                      3-3       85
                                      1-4       70
                                      4-3       62
                                      3-4       56
                                      4-1       45
                                      3-0       43
                                      0-3       42
                                      3-5       19
                                      2-5       16
                                      5-2       13
                                      5-3       11
                                      4-4        5
                                      5-1        4
                                      0-4        3
                                      1-5        3
                                      5-4        3
                                      2-6        2
                                      6-3        2
                                      3-6        1
                                      4-0        1
                                      4-5        1
(GA2, 15/06/2018, 13:00, EGY, URU, A) 1-1     3209
                                      2-1     3045
                                      0-0     2565
                                      1-0     2378
                                      1-2     2075
                                      0-1     1616
                                      3-1      899
                                      2-2      784
                                      3-2      776
                                      2-0      630
                                      2-3      499
                                      1-3      370
                                      0-2      295
                                      4-2      225
                                      4-1      106
                                      2-4       93
                                      3-0       88
                                      3-3       85
                                      4-3       64
                                      3-4       41
                                      1-4       38
                                      5-2       28
                                      5-3       26
                                      0-3       20
                                      5-1        9
                                      3-5        7
                                      1-5        6
                                      5-4        6
                                      2-5        5
                                      4-0        4
                                      4-6        2
                                      4-7        2
                                      6-2        2
                                      6-3        2
(GA3, 19/06/2018, 19:00, RUS, EGY, A) 1-1     3330
                                      1-2     3067
                                      0-0     2567
                                      0-1     2379
                                      2-1     2004
                                      1-0     1522
                                      1-3      904
                

In [47]:
#test = df.groupby('match')['score'].value_counts()
#tester = test.groupby('match').idxmax()
#display(tester.loc[tester.score.max()])
#test1 = pd.DataFrame(test)

#grouped = df.groupby('match')
#display(grouped['team1Score'])

#pd.set_option('display.max_rows', 1000)
#for i in test:
#    print(i)



NameError: name 'groupresults' is not defined

In [45]:
import numpy as np
scores = df.groupby('match')[['team1Score', 'team2Score']]

most_freq = pd.DataFrame()
for i in df.match.unique():
    scores = df[['team1Score', 'team2Score']]
    scores, counts = np.unique(scores.values, return_counts=True, axis=0)
    scores = [str(x) for x in scores]
    freq = counts / 20000
    most_freq1 = most_freq.append(pd.DataFrame({'score': scores, 'count': counts, 'freq': freq}, columns=['score', 'count', 'freq']))
    
    
most_freq

""


In [23]:
for i in range(5):
    records, results = runGroupStage(schedule, predictions, teams)
    print(records)
    for rnd in ['AF','QF','SF','FN']:
        roundresults, results = runEliminationRound(rnd, results, schedule, predictions)
        print(roundresults)
gameId='GC2', date='16/06/2018', time='17:00', team1='PER', team2='DEN', group='C'

ValueError: too many values to unpack (expected 2)

In [9]:
records, results = runGroupStage(schedule, predictions, teams)
groupstages = pd.DataFrame(records)
#groupstages = groupstages.sort_values(by=['group', 'p'], ascending=['False', 'False'])
groupstages
groupstages = pd.DataFrame(records)
temp = pd.DataFrame()
temp['id'] = 0
for i in range(10):
    records, results = runGroupStage(schedule, predictions, teams)
    temp = temp.append(records)
    for j in temp['match']:
        if (temp['team1Score'] not in groupstages['team1Score'].values and temp['team2Score'] not in groupstages['team1Score']):
            temp['id'] += 1
        else:
            temp['id'] = temp['id']
    #groupstages = groupstages.append(pd.DataFrame(records))
    
#groupstages.groupby('match')[['team1Score', 'team2Score']].mean()

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


ValueError: Lengths must match to compare

In [3]:
import pandas as pd
worldcup_winnerfin = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresultsfinal.csv')
display(worldcup_winnerfin.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner23 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults23.csv')
display(worldcup_winner23.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner22 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults22.csv')
display(worldcup_winner22.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner21 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults21.csv')
display(worldcup_winner21.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner20 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults20.csv')
display(worldcup_winner20.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner19 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults19.csv')
display(worldcup_winner19.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner18 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults18.csv')
display(worldcup_winner18.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner17 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults17.csv')
display(worldcup_winner17.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner16 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults16.csv')
display(worldcup_winner16.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner15 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults15.csv')
display(worldcup_winner15.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner14 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults14.csv')
display(worldcup_winner14.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner13 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults13.csv')
display(worldcup_winner13.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner12 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults12.csv')
display(worldcup_winner12.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner11 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults11.csv')
display(worldcup_winner11.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner10 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults10.csv')
display(worldcup_winner10.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner9 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults9.csv')
display(worldcup_winner9.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner8 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults8.csv')
display(worldcup_winner8.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

worldcup_winner7 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults7.csv')
display(worldcup_winner7.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))


,winner,iteration
12,GER,3696
3,BRA,3588
10,ESP,2556
27,SUI,1890
23,POR,1859
9,ENG,1234
11,FRA,1030
2,BEL,1015
26,SRB,648
13,IRN,496


,winner,iteration
3,BRA,18285
12,GER,14149
10,ESP,11470
11,FRA,7822
0,ARG,6853
5,CRC,6466
9,ENG,5725
19,MEX,3161
1,AUS,3114
20,NGA,2056


,winner,iteration
19,MEX,3634
0,ARG,1823
9,ENG,1733
3,BRA,1711
5,CRC,1629
12,GER,1405
30,TUN,811
16,KOR,760
10,ESP,719
15,JPN,698


,winner,iteration
20,NGA,4584
16,KOR,3544
21,PAN,2253
15,JPN,2243
25,RUS,981
18,MAR,872
27,SRB,706
30,TUN,650
26,SEN,488
5,CRC,383


,winner,iteration
3,BRA,2167
12,GER,2152
28,SUI,1095
19,MEX,991
4,COL,985
9,ENG,978
10,ESP,871
11,FRA,856
15,JPN,829
23,POL,819


...

,winner,iteration
12,GER,1351
9,ENG,891
2,BEL,890
3,BRA,846
28,SUI,730
15,JPN,530
6,CRO,425
11,FRA,395
27,SRB,385
23,POL,363


,winner,iteration
12,GER,2416
3,BRA,2121
28,SUI,1545
2,BEL,1218
9,ENG,1119
24,POR,1080
10,ESP,985
27,SRB,895
11,FRA,856
13,IRN,747


,winner,iteration
3,BRA,2088
12,GER,2012
28,SUI,1582
24,POR,1261
9,ENG,1114
2,BEL,1092
10,ESP,1070
11,FRA,1000
27,SRB,919
13,IRN,731


,winner,iteration
12,GER,2207
3,BRA,2147
28,SUI,1505
24,POR,1236
2,BEL,1061
10,ESP,1056
9,ENG,1040
27,SRB,968
11,FRA,914
13,IRN,841


,winner,iteration
3,BRA,2123
12,GER,2042
28,SUI,1296
24,POR,1078
9,ENG,1043
27,SRB,993
2,BEL,988
10,ESP,929
11,FRA,828
13,IRN,810


In [188]:
worldcup_winner = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults.csv')
worldcup_winner2 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults2.csv')
worldcup_winner3 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults3.csv')
worldcup_winner4old = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults4old.csv')
worldcup_winner5 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults5.csv')
worldcup_winner6 = pd.read_csv('C:\\Data_Science\\World Cup 2018\\FIFA real world stats\\simresults6.csv')
display(worldcup_winner6.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

#display(worldcup_winner5.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))
#display(worldcup_winner4old.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))
#display(worldcup_winner.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))
#display(worldcup_winner2.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))
#display(worldcup_winner3.groupby('winner', as_index=False).count().sort_values('iteration',ascending=False))

,winner,iteration
25,RUS,2425
24,POR,1988
28,SUI,1721
3,BRA,1512
8,EGY,1433
10,ESP,1398
11,FRA,1233
12,GER,1050
1,AUS,963
31,URU,715


,winner,iteration
11,FRA,1727
24,POR,1655
28,SUI,1601
3,BRA,1529
10,ESP,1481
12,GER,1281
9,ENG,895
5,CRC,795
13,IRN,765
31,URU,683
